# Code to find non-unique Minimal Sum representations

Observe: 

-Only non-unique for >7 players

-sum must always be the same

-differences at most +1/-1 



## use italy as example

In [30]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import os
import glob
import time 
import numpy as np
from scipy import optimize
from itertools import combinations
from mwc_class import getMVWs
from mwc_functions import *
from optimization_functions import *


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
start_time=time.time()
italy_data = getMVWs('italy.csv', name='italy', save_results=False,verify_mwcs=True)
prelims = italy_data.preliminaries()
country_MIW = italy_data.minimal_voting_weights_pipeline()
end_time=time.time()
duration_Verify=end_time-start_time

print(duration_Verify)


8.106972932815552


idea: 
Function takes in weights and creates all possible combinations of potential equivalent representations
passes each one to the linear constraints df: 
    - adds weights as upper and lower bounds 
tests whether the optimization works with any of these additional constraints
- potentially very time consuming

But: LinearConstraint Elements not manipulatable... 

In [52]:
type(mvw)

NameError: name 'mvw' is not defined

In [31]:
def collect_all_representations(mvw, year, n_in_year, constraints_df):
    weights = mvw
    alternative_weights = possible_other_weights(weights)
    optimal_weights = []
    total_weights = len(alternative_weights)
    one_percent = round(total_weights / 100)
    start_time= time.time()
    for i, alt_weights in enumerate(alternative_weights):
        lin_cons = alt_lin_cons(alt_weights, constraints_df)
        mvw_alt = get_min_vote_weights(year, n_in_year, lin_cons)
        if mvw_alt.status == 0:
            optimal_weights.append(alt_weights)
        if (i + 1) % one_percent == 0:
            current_time = time.time()
            elapsed_time = current_time - start_time
            print(f"Tested {((i + 1) / total_weights) * 100:.2f}% of {total_weights} elements in {elapsed_time:.2f} seconds")
    return optimal_weights    
    

In [13]:
italy_data.all_lin_cons.get('1987')

In [32]:
def alt_lin_cons(weights, constraints_df):
    A = constraints_df.to_numpy()
    if isinstance(weights, list): # necessary from output of possible_other_weights function 
        weights = weights[0]
    lbnd = np.zeros(len(constraints_df))  
    lbnd[:len(weights)] = weights  
    lbnd[len(weights):] = 1  
    upbnd = np.full(len(constraints_df), np.inf) 
    upbnd[:len(weights)] = weights  
    lin_cons = LinearConstraint(A, lbnd, upbnd)

    return lin_cons

In [16]:
weights=np.array([1,2,3,4,0])

In [18]:
pairs = list(combinations(range(len(weights)), 2))
print(pairs)

[(0, 1), (0, 2), (0, 3), (0, 4), (1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4)]


In [33]:
def possible_other_weights(weights):
    weight = np.round(weights).astype(int)
    
    # Get unique pairs to be changed +1/-1
    pairs = list(combinations(range(len(weight)), 2)) # get all possbible pairs of weights 
    
    alt_weights = []
    
    for i, j in pairs:
        for change in [(1, -1), (-1, 1)]:
            new_weights = weight.copy()
            if new_weights[i] + change[0] >= 0 and new_weights[j] + change[1] >= 0:
                new_weights[i] += change[0]
                new_weights[j] += change[1]
                if not any(np.array_equal(new_weights, arr) for arr in alt_weights):
                    alt_weights.append(new_weights)
    alt_weights.append(weight)
    return alt_weights

In [21]:
alt_weights=possible_other_weights(weights)

In [23]:
print(alt_weights)

[array([2, 1, 3, 4, 0]), array([0, 3, 3, 4, 0]), array([2, 2, 2, 4, 0]), array([0, 2, 4, 4, 0]), array([2, 2, 3, 3, 0]), array([0, 2, 3, 5, 0]), array([0, 2, 3, 4, 1]), array([1, 3, 2, 4, 0]), array([1, 1, 4, 4, 0]), array([1, 3, 3, 3, 0]), array([1, 1, 3, 5, 0]), array([1, 1, 3, 4, 1]), array([1, 2, 4, 3, 0]), array([1, 2, 2, 5, 0]), array([1, 2, 2, 4, 1]), array([1, 2, 3, 3, 1])]


In [34]:
cons = italy_data.all_constraints.get('2006')
lin_cons=get_lin_cons(cons)
min_weights=get_min_vote_weights('2006',italy_data.n_in_year,lin_cons)

In [50]:
italy_data.n_in_year

{'1987': 14,
 '1992': 16,
 '1994': 14,
 '1996': 16,
 '2001': 11,
 '2006': 17,
 '2008': 8,
 '2013': 10,
 '2018': 7}

In [38]:
print(min_weights.x)

[  1.   1.  71.  16.  96.  17.   1.  26.   4.   6.  41.  18.   4. 176.
  39.  10.  15.]


In [63]:
alt_weights=collect_all_representations(min_weights, year='2006', n_in_year = italy_data.n_in_year, constraints_df = cons)

Tested 1.10% of 273 elements in 1.43 seconds
Tested 2.20% of 273 elements in 2.80 seconds
Tested 3.30% of 273 elements in 4.23 seconds
Tested 4.40% of 273 elements in 5.65 seconds
Tested 5.49% of 273 elements in 7.04 seconds
Tested 6.59% of 273 elements in 8.40 seconds
Tested 7.69% of 273 elements in 9.79 seconds
Tested 8.79% of 273 elements in 11.17 seconds
Tested 9.89% of 273 elements in 12.54 seconds
Tested 10.99% of 273 elements in 13.89 seconds
Tested 12.09% of 273 elements in 15.24 seconds
Tested 13.19% of 273 elements in 16.61 seconds
Tested 14.29% of 273 elements in 17.98 seconds
Tested 15.38% of 273 elements in 19.32 seconds
Tested 16.48% of 273 elements in 20.68 seconds
Tested 17.58% of 273 elements in 22.03 seconds
Tested 18.68% of 273 elements in 23.39 seconds
Tested 19.78% of 273 elements in 24.79 seconds
Tested 20.88% of 273 elements in 26.20 seconds
Tested 21.98% of 273 elements in 27.57 seconds
Tested 23.08% of 273 elements in 28.97 seconds
Tested 24.18% of 273 elements

In [64]:
alt_weights

[array([  1,   1,  71,  16,  96,  17,   1,  26,   4,   6,  41,  18,   4,
        176,  39,  10,  15])]

 function finds at least the correct value --> one way it works :) 
 test now with a number of seats surely having multiple representations 

In [27]:
#test the following vectors: 15,13,10,8,6,4,4,2,1
#this is an example from Directed and weighted majority games, 1995

min_Seats_known = {
    2001: {'A': 15, 'B': 13, 'C': 10, 'D': 8, 'E': 6, 'F': 4, 'G': 4, 'H': 2, 'I': 1},
    2002: {'A': 17, 'B': 15, 'C': 11, 'D': 9, 'E': 7, 'F': 5, 'G': 4, 'H': 2, 'I': 1},
    2003: {'A': 13, 'B': 7, 'C': 6, 'D': 6, 'E': 4, 'F': 4, 'G': 4, 'H': 3, 'I': 2},
}

# create some variety
min_Seats_known[2004] = {party: seats * 5 for party, seats in min_Seats_known[2001].items()}
min_Seats_known[2005] = {party: seats * 3 for party, seats in min_Seats_known[2002].items()}
min_Seats_known[2006] = {party: seats * 4 for party, seats in min_Seats_known[2003].items()}

# Prepare df 
data = []
for year in range(2001, 2007):
    for party, seats in min_Seats_known[year].items():
        data.append({'Party': party, 'Date': year, '# of Seats': seats})

krohn_test_data = pd.DataFrame(data)
krohn_test_data['Date']=krohn_test_data['Date'].apply(str)
krohn_test_data.to_csv('data/krohn.csv',index=False)



In [40]:
kurz_test=read_csv_to_dataframe('kurz_2012_example.csv',encoding='UTF-8',delimiter=',')

In [41]:
kurz_test_1=transform_and_sort_dataframe(kurz_test_df)

In [42]:
kurz_test_1.head()

,Year,Party,# of Seats
0,2001,A,23
1,2001,B,15
2,2001,C,13
3,2001,D,11
4,2001,E,9


In [9]:
start_time=time.time()
kurz_data = getMVWs('kurz_2012_example.csv', name='kurz_2012_example', save_results=True,verify_mwcs=True,encoding='UTF-8',delimiter=',')
prelims = kurz_data.preliminaries()
country_MIW = kurz_data.minimal_voting_weights_pipeline()
end_time=time.time()
duration_Verify=end_time-start_time

print(duration_Verify)

0.5343070030212402


results indicate optimizer always finds "indicated result" however I dont really know how that can be, probably due to this case having changes across equivalence classes?

In [49]:
kurz_data.all_min_weights

,w_A,w_B,w_C,w_D,w_E,w_F,w_G,w_H,w_I
0,1,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...
816,0,-1,1,1,0,0,-1,0,0
817,0,0,0,0,0,1,-1,0,0
818,0,0,0,0,1,0,-1,0,0
819,0,0,0,1,0,0,-1,0,0


In [54]:
alt_weights=collect_all_representations(kurz_data.all_min_weights.get('2001'), year='2001', n_in_year = kurz_data.n_in_year, constraints_df = kurz_data.all_constraints.get('2001'))

Tested 1.37% of 73 elements in 0.00 seconds
Tested 2.74% of 73 elements in 0.00 seconds
Tested 4.11% of 73 elements in 0.01 seconds
Tested 5.48% of 73 elements in 0.01 seconds
Tested 6.85% of 73 elements in 0.01 seconds
Tested 8.22% of 73 elements in 0.01 seconds
Tested 9.59% of 73 elements in 0.01 seconds
Tested 10.96% of 73 elements in 0.01 seconds
Tested 12.33% of 73 elements in 0.01 seconds
Tested 13.70% of 73 elements in 0.01 seconds
Tested 15.07% of 73 elements in 0.01 seconds
Tested 16.44% of 73 elements in 0.01 seconds
Tested 17.81% of 73 elements in 0.01 seconds
Tested 19.18% of 73 elements in 0.01 seconds
Tested 20.55% of 73 elements in 0.02 seconds
Tested 21.92% of 73 elements in 0.02 seconds
Tested 23.29% of 73 elements in 0.02 seconds
Tested 24.66% of 73 elements in 0.02 seconds
Tested 26.03% of 73 elements in 0.02 seconds
Tested 27.40% of 73 elements in 0.02 seconds
Tested 28.77% of 73 elements in 0.02 seconds
Tested 30.14% of 73 elements in 0.02 seconds
Tested 31.51% of 

In [55]:
(alt_weights)

[array([23, 15, 13, 11,  9,  8,  3,  2,  2])]

In [56]:
test=possible_other_weights(kurz_data.all_min_weights.get('2001'))

In [57]:
(test)

[array([24, 14, 13, 11,  9,  8,  3,  2,  2]),
 array([22, 16, 13, 11,  9,  8,  3,  2,  2]),
 array([24, 15, 12, 11,  9,  8,  3,  2,  2]),
 array([22, 15, 14, 11,  9,  8,  3,  2,  2]),
 array([24, 15, 13, 10,  9,  8,  3,  2,  2]),
 array([22, 15, 13, 12,  9,  8,  3,  2,  2]),
 array([24, 15, 13, 11,  8,  8,  3,  2,  2]),
 array([22, 15, 13, 11, 10,  8,  3,  2,  2]),
 array([24, 15, 13, 11,  9,  7,  3,  2,  2]),
 array([22, 15, 13, 11,  9,  9,  3,  2,  2]),
 array([24, 15, 13, 11,  9,  8,  2,  2,  2]),
 array([22, 15, 13, 11,  9,  8,  4,  2,  2]),
 array([24, 15, 13, 11,  9,  8,  3,  1,  2]),
 array([22, 15, 13, 11,  9,  8,  3,  3,  2]),
 array([24, 15, 13, 11,  9,  8,  3,  2,  1]),
 array([22, 15, 13, 11,  9,  8,  3,  2,  3]),
 array([23, 16, 12, 11,  9,  8,  3,  2,  2]),
 array([23, 14, 14, 11,  9,  8,  3,  2,  2]),
 array([23, 16, 13, 10,  9,  8,  3,  2,  2]),
 array([23, 14, 13, 12,  9,  8,  3,  2,  2]),
 array([23, 16, 13, 11,  8,  8,  3,  2,  2]),
 array([23, 14, 13, 11, 10,  8,  3

In [64]:
lin_cons=alt_lin_cons(test[-5:-4],kurz_data.all_constraints.get('2001'))

In [74]:
mvw_alt = get_min_vote_weights('2001',kurz_data.n_in_year, lin_cons)

In [75]:
(mvw_alt)

        message: The problem is infeasible. (HiGHS Status 8: model_status is Infeasible; primal_status is At lower/fixed bound)
        success: False
         status: 2
            fun: None
              x: None
 mip_node_count: None
 mip_dual_bound: None
        mip_gap: None

In [76]:
A = kurz_data.all_constraints.get('2001').to_numpy()
if isinstance(test[-5:-4], list): # necessary from output of possible_other_weights function 
    weights = test[-5:-4][0]
lbnd = np.zeros(len(kurz_data.all_constraints.get('2001')))  
#lbnd[:len(weights)] = weights  
lbnd[len(weights):] = 1  
upbnd = np.full(len(kurz_data.all_constraints.get('2001')), np.inf) 
upbnd[:len(weights)] = weights  

In [77]:
lin_cons = LinearConstraint(A, lbnd, upbnd)

In [78]:
mvw_alt = get_min_vote_weights('2001',kurz_data.n_in_year, lin_cons)

In [79]:
(mvw_alt)

        message: The problem is infeasible. (HiGHS Status 8: model_status is Infeasible; primal_status is At lower/fixed bound)
        success: False
         status: 2
            fun: None
              x: None
 mip_node_count: None
 mip_dual_bound: None
        mip_gap: None

In [80]:
upbnd

array([23., 15., 13., 11.,  9.,  8.,  4.,  2.,  1., inf, inf, inf, inf,
       inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, in

In [73]:
A[0:9]

array([[1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1]])

this apporach seems not to be working, milp does not find alternative solutions ? 

In [ ]:
def verify_coals(all_optimized_Seats,winning_coal_dict): 
    test_dict={}
    errors= {}
    
    for year, yearly_matching in all_optimized_Seats.items(): 
            yearly_optimized_Seats= yearly_matching
            yearly_mw_winning_dict = help_test_mvws(yearly_optimized_Seats)
            test_boolean,wrong_coals = test_mvws(year,winning_coal_dict,yearly_mw_winning_dict)
            test_dict[year]=test_boolean
            errors[year]=wrong_coals
    return test_dict,errors
def help_test_mvws(optimized_seats):
    '''helper function for test_mvws'''
    ## creates dict just like winning_coal_dict but from mvw´s, used later to ensure equivalency of games 
    ## takes in dict from mvw_to_parties and uses same logic as coalition_combinatorics_generator and win_coals but for dicts
    
    mw_winning_coal_dict = {}
    total_seats = sum(optimized_seats.values()) #new Q
    parties = list(optimized_seats.keys())

    # Generate all unique coalitions
    mw_winning_coal_dict[''] = 0 #add empty coal manually 
    for r in range(1, len(parties) + 1):
        for combo in itertools.combinations(parties, r):
            coalition = '+'.join(combo) #name of coalition 
            coalition_seats = sum(optimized_seats[party] for party in combo) #seats is sum of their weights 
            mw_winning_coal_dict[coalition] = 1 if coalition_seats > (total_seats / 2) else 0 #indicate winning or losing (again, with strict inequality)

    return mw_winning_coal_dict


def test_mvws(year, winning_coal_dict, mw_winning_coal_dict):
    ## takes in mw_winning_coal_dict and compares it to winning_coal_dict[year] 
    ## if the set of coalitions is the same and if all coalitions have the same value, the game is identical and thus the mvs represent the same game as the original parliament 
    ## returns boolean as well as dict with errors:
    winning_coals_by_year = {coalition: value for (yr, coalition), value in winning_coal_dict.items() if yr == year} # bit convoluted since I need 'year' twice, basically just separates the relevant year from the winning_coal_dict
    errors = {}
    # Compare 
    for coalition, value in winning_coals_by_year.items():
        if coalition not in mw_winning_coal_dict or mw_winning_coal_dict[coalition] != value:
            errors[(year,coalition)]=(value,mw_winning_coal_dict.get(coalition,'not in here')) #writes to a coalition the value from the winning coal and the min_winning_coal (or indicates that the coal is not actually mi-w)
    check = len(errors)==0
    return check, errors

In [6]:
test_dict={}
test_dict['A']=23
test_dict['B']=15
test_dict['C']=13
test_dict['D']=11
test_dict['E']=9
test_dict['F']=8
test_dict['G']=4
test_dict['H']=2
test_dict['I']=1

In [25]:
alt_dict=help_test_mvws(test_dict)
check,errors =test_mvws('2001',kurz_data.winning_coal_dict,alt_dict)
(errors)

{('2001', 'A+B+G+H'): (0, 1),
 ('2001', 'A+D+F+I'): (1, 0),
 ('2001', 'A+E+F+G'): (0, 1),
 ('2001', 'A+E+F+H+I'): (1, 0)}

In [26]:
kurz_data.totalseats_in_year

{'2001': 86, '2002': 86, '2003': 86, '2004': 430, '2005': 258, '2006': 344}

test the kron data set: 

In [28]:
start_time=time.time()
krohn_data = getMVWs('krohn.csv', name='krohn_example', save_results=True,verify_mwcs=True,encoding='UTF-8',delimiter=',')
prelims = krohn_data.preliminaries()
country_MIW = krohn_data.minimal_voting_weights_pipeline()
end_time=time.time()
duration_Verify=end_time-start_time

print(duration_Verify)

0.6056113243103027


In [38]:
alt_weights=collect_all_representations(krohn_data.all_min_weights.get('2003'), year='2003', n_in_year = krohn_data.n_in_year, constraints_df = krohn_data.all_constraints.get('2003'))

Tested 1.37% of 73 elements in 0.00 seconds
Tested 2.74% of 73 elements in 0.01 seconds
Tested 4.11% of 73 elements in 0.01 seconds
Tested 5.48% of 73 elements in 0.01 seconds
Tested 6.85% of 73 elements in 0.01 seconds
Tested 8.22% of 73 elements in 0.01 seconds
Tested 9.59% of 73 elements in 0.02 seconds
Tested 10.96% of 73 elements in 0.02 seconds
Tested 12.33% of 73 elements in 0.02 seconds
Tested 13.70% of 73 elements in 0.02 seconds
Tested 15.07% of 73 elements in 0.02 seconds
Tested 16.44% of 73 elements in 0.02 seconds
Tested 17.81% of 73 elements in 0.03 seconds
Tested 19.18% of 73 elements in 0.03 seconds
Tested 20.55% of 73 elements in 0.03 seconds
Tested 21.92% of 73 elements in 0.03 seconds
Tested 23.29% of 73 elements in 0.03 seconds
Tested 24.66% of 73 elements in 0.03 seconds
Tested 26.03% of 73 elements in 0.04 seconds
Tested 27.40% of 73 elements in 0.04 seconds
Tested 28.77% of 73 elements in 0.04 seconds
Tested 30.14% of 73 elements in 0.04 seconds
Tested 31.51% of 

In [39]:
(alt_weights)

[array([13,  7,  6,  6,  4,  4,  4,  3,  2]),
 array([13,  7,  6,  6,  4,  4,  4,  2,  3])]